In [63]:
import datasets 
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary') 
df = dataset['train'].to_pandas()
df=df[['hatespeech',
 'text',
 'respect',
 'insult']]


Using custom data configuration ucberkeley-dlab--measuring-hate-speech-c32713cabe528196
Found cached dataset parquet (/Users/animeshsengupta/.cache/huggingface/datasets/ucberkeley-dlab___parquet/ucberkeley-dlab--measuring-hate-speech-c32713cabe528196/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
import pandas as pd
import numpy as np
import json
import torch

from datasets import Dataset, load_metric
from transformers import DistilBertTokenizerFast, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainerCallback, TrainingArguments
from torch.utils.data import DataLoader


from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report

BASE_MODEL = "distilbert-base-uncased"
LEARNING_RATE = 1e-4
MAX_LENGTH = 120
BATCH_SIZE = 16
EPOCHS = 50

In [10]:
tokenizer = DistilBertTokenizerFast.from_pretrained(BASE_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier

In [47]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
cols_to_dummy=["hatespeech","insult","respect"]
df = pd.get_dummies(data=df,columns=cols_to_dummy)



In [62]:
dum_col=[]
l=[[col for col in df if col.startswith(x)] for x in cols_to_dummy]
flat_list = [item for sublist in l for item in sublist]
df["labels"]=df[flat_list].values.tolist()
df=df.drop(columns=flat_list,axis=1)
df.head(5)

,text,labels
0,Yes indeed. She sort of reminds me of the elde...,"[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
1,The trans women reading this tweet right now i...,"[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
2,Question: These 4 broads who criticize America...,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]"
3,It is about time for all illegals to go back t...,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]"
4,For starters bend over the one in pink and kic...,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]"


In [65]:
def preprocess(df,cols_to_dummy,tokenizer):
    df=pd.get_dummies(data=df,columns=cols_to_dummy)
    dum_col_list=[[col for col in df if col.startswith(x)] for x in cols_to_dummy]
    dum_col = [item for sublist in dum_col_list for item in sublist]
    df["labels"]=df[dum_col].values.tolist()
    df[["input_ids","attention_mask"]]=df.apply(lambda row: pd.Series(tokenizer(row['text'], truncation=True, padding="max_length", max_length=MAX_LENGTH)),axis=1)
    df=df.drop(columns=dum_col,axis=1)
    return df

In [66]:
df=preprocess(df,cols_to_dummy,tokenizer)

In [67]:
df.head(5)

,text,labels,input_ids,attention_mask
0,Yes indeed. She sort of reminds me of the elde...,"[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[101, 2748, 5262, 1012, 2016, 4066, 1997, 1553...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,The trans women reading this tweet right now i...,"[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[101, 1996, 9099, 2308, 3752, 2023, 1056, 2839...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ..."
2,Question: These 4 broads who criticize America...,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]","[101, 3160, 1024, 2122, 1018, 5041, 2015, 2040...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,It is about time for all illegals to go back t...,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]","[101, 2009, 2003, 2055, 2051, 2005, 2035, 6206...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,For starters bend over the one in pink and kic...,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]","[101, 2005, 29400, 8815, 2058, 1996, 2028, 199...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
